In [ ]:
import os
import glob
import librosa
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from librosa import display
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
def data_filtering(file_name):
    print('file name: ', file_name)
    try:
        audio, sample_rate = librosa.load(file_name, res_type = 'kaiser_best')
        mcu_sr = 256
        audio_mcu = librosa.resample(audio, sample_rate, mcu_sr, res_type = 'kaiser_best')
        audio_mcu = audio_mcu[128:128+mcu_sr]
        if len(audio_mcu) != mcu_sr:
            return None
        fft = np.fft.fft(audio_mcu)
        spectrum = np.abs(fft)
        final = spectrum[:int(len(spectrum)/2)]
        print(final.shape)
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        print(e);
        return None
    return final

In [ ]:
fulldatasetpath = './audio/'
metadata = pd.read_csv('./metadata/UrbanSound8K.csv')
features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    class_label = row["classID"]
    data = data_filtering(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

In [ ]:
copy = featuresdf.dropna(axis = 0)

In [ ]:
# 피클로 데이터 저장
copy.to_pickle("./data/data_final.pkl")

In [ ]:
data_filtered = pd.read_pickle("./data/data_final.pkl")

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
X = np.array(data_filtered.feature.tolist())
y = np.array(data_filtered.class_label.tolist())

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [ ]:
X = np.array(X, dtype = float)
yy = np.array(yy, dtype = float)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.3, random_state = 100)

# TensorFlow

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation = 'softmax')
])

num_batch_size = 128
learning_rate = 0.001
opt = keras.optimizers.Adam(learning_rate = learning_rate)

model.compile(optimizer = opt,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
checkpoint_path = './plz/cp-{epoch:03d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

class callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, eopch, logs={}):
        if(logs.get('accuracy') > 0.95) :
            print("\n----reach 95% accuracy, stop training----")
            self.model.stop_training = True
            
callbacks = callback()

In [ ]:
model.fit(x_train,
          y_train,
          batch_size=num_batch_size,
          epochs = 200,
          callbacks = [callbacks])

In [ ]:
print('\n# Evaluate on test data')
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)

In [ ]:
model.summary()

In [ ]:
from tinymlgen import port

In [ ]:
if __name__ == '__main__':
    c_code = port(model, variable_name = 'sound_model', pretty_print = True, optimize = False)
print(c_code)

In [ ]:
model.save("./final/real_final/model_final.h5")

# Visualization

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [ ]:
metadata = pd.read_csv('./metadata/UrbanSound8K.csv')

In [ ]:
# draw confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues,
                          percentage = False):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize = (8,8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
model = keras.models.load_model('./final/real_final/model_final.h5')

In [ ]:
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

In [ ]:
def transform_1D(y):
    tmp = []
    for line in y:
        maxi = max(line)
        index = list(line).index(maxi)
        tmp.append(index)
    return tmp

In [ ]:
y_train = np.array(transform_1D(y_train), dtype=np.float32)
y_test = np.array(transform_1D(y_test), dtype=np.float32)

In [ ]:
y_pred_train = np.array(transform_1D(y_pred_train), dtype=np.float32)
y_pred_test = np.array(transform_1D(y_pred_test), dtype=np.float32)

In [ ]:
class_names = metadata.groupby(['class', 'classID'], as_index = False).sum()['class']
plot_confusion_matrix(y_train, y_pred_train, classes=class_names, cmap=plt.cm.Greens,\
                      normalize= True)
plt.show()

In [ ]:
class_names = metadata.groupby(['class', 'classID'], as_index = False).sum()['class']
plot_confusion_matrix(y_test, y_pred_test, classes=class_names, cmap=plt.cm.Greens,\
                      normalize= True)
plt.show()

# 데이터 확인

In [ ]:
audio, sample_rate = librosa.load('./audio/fold1/7061-6-0-0.wav', res_type='kaiser_best')

In [ ]:
audio = librosa.resample(audio, sample_rate, 256)

In [ ]:
fig_size = (15, 10)
plt.figure(figsize = fig_size)
plt.plot(audio, alpha = 0.4)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.title("Power spectrum")

In [ ]:
fig_size = (15, 10)
plt.figure(figsize = fig_size)
plt.plot(np.abs(np.fft.fft(audio)[:128]), alpha = 0.4)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.title("Power spectrum")

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_rf = RandomForestClassifier(n_estimators=100, bootstrap=True,\
                                  max_depth=None,max_leaf_nodes=None,\
                                  max_features='auto')

In [ ]:
model_rf.fit(x_train, y_train)

In [ ]:
print(model_rf.score(x_train, y_train))
print(model_rf.score(x_test, y_test))

# SVM

In [ ]:
import sklearn.svm as svm

In [ ]:
model_svm = svm.SVC(kernel = 'rbf', C = 1000000)

In [ ]:
data_svm = pd.read_pickle('./data/data_final.pkl')

In [ ]:
X = data_svm['feature']
y = data_svm['class_label']

In [ ]:
X = np.array(X.tolist(), dtype = np.float64)
y = np.array(y.tolist(), dtype = np.float64)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
model_svm.fit(x_train, y_train)

In [ ]:
y_pred = model_svm.predict(x_test)
train_acc = model_svm.score(x_train, y_train)
test_acc = model_svm.score(x_test, y_test)
print('Training Accuracy:', train_acc)
print('Test Accuracy:', test_acc)